In [ ]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [ ]:
def processAndWriteDf(df):
    df_cropped = df.iloc[:,1:]
    print(df_cropped.head(5))
    df_cropped.to_csv("./submission.csv",index = False)
    print("write done")
    return df_cropped

In [ ]:
def getPredictionFromTPUModel():
    model = tf.keras.models.load_model('../input/plant-pathology-2020-tpu/my_modelv6.h5')
    base_dir = '../input/plant-pathology-2020-fgvc7/images/'
    test_csv = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
    test_csv["image_id"] = test_csv["image_id"]+'.jpg'
    #test_csv = test_csv[:4]
    test_datagen = ImageDataGenerator(rescale=1./255)
    batchSize = 2

    test_generator = test_datagen.flow_from_dataframe(
            test_csv,
            directory = base_dir,
            x_col = 'image_id',
            y_col = [],
            target_size=(1365,2048),
            batch_size=batchSize,
            class_mode=None,
            shuffle = False
            )

    p = model.predict(test_generator)
    test_csv = pd.read_csv('../input/plant-pathology-2020-fgvc7/test.csv')
    p_df = pd.DataFrame(p,columns=['healthy','multiple_diseases','rust','scab'])
    result = pd.concat([test_csv, p_df], axis=1)
    return result

In [ ]:
isTPU = True

if isTPU:
    df = getPredictionFromTPUModel()
    print(df.head(5))
    df.to_csv("./submission.csv",index = False)
    print("write done")
else:
    df = pd.read_csv("../input/notebook45bc751087/submission.csv")
    df.head()
    result = processAndWriteDf(df)
    result.head(5)